In [1]:
#impoting libraries
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
#shape of the data
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
#check and prepare data if needed
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
# used a normalized version of the data different than Step A
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

In [11]:
#define regresion model using keras
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

# build the model
model = regression_model()

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [13]:
# fit the model
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)



Epoch 1/50
 32/721 [>.............................] - ETA: 3s - loss: 58014.4414

2024-01-22 19:16:10.304915: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-01-22 19:16:10.314739: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394335000 Hz
2024-01-22 19:16:10.315509: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5563626b3f10 executing computations on platform Host. Devices:
2024-01-22 19:16:10.315571: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-01-22 19:16:10.379413: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

721/721 [==============================] - 0s 343us/step - loss: 34742.3364
Epoch 2/50
721/721 [==============================] - 0s 90us/step - loss: 6012.5732
Epoch 3/50
721/721 [==============================] - 0s 104us/step - loss: 2422.4498
Epoch 4/50
721/721 [==============================] - 0s 107us/step - loss: 2214.7478
Epoch 5/50
721/721 [==============================] - 0s 87us/step - loss: 1983.5541
Epoch 6/50
721/721 [==============================] - 0s 76us/step - loss: 1775.5449
Epoch 7/50
721/721 [==============================] - 0s 61us/step - loss: 1581.3025
Epoch 8/50
721/721 [==============================] - 0s 59us/step - loss: 1404.9639
Epoch 9/50
721/721 [==============================] - 0s 74us/step - loss: 1246.6117
Epoch 10/50
721/721 [==============================] - 0s 59us/step - loss: 1113.9174
Epoch 11/50
721/721 [==============================] - 0s 63us/step - loss: 987.1567
Epoch 12/50
721/721 [==============================] - 0s 58us/step - l

In [14]:
#evaluate the model
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 68us/step


118.40734927476802

In [15]:
# reporting MAS
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

118.4073688091752 0.0


In [16]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("StD: "+str(standard_deviation))

MSE 1: 94.17769793167855
MSE 2: 87.6678263840166
MSE 3: 67.56737776327287
MSE 4: 80.88573011379798
MSE 5: 73.78214078736536
MSE 6: 90.46124978667324
MSE 7: 78.47813129116416
MSE 8: 69.89055278000322
MSE 9: 62.32774513516225
MSE 10: 60.37829054057791
MSE 11: 55.57381846603838
MSE 12: 54.93813127684362
MSE 13: 58.876590704069166
MSE 14: 50.973197887630526
MSE 15: 52.95824068186738
MSE 16: 51.74090732339902
MSE 17: 44.79778561391491
MSE 18: 48.44487366167087
MSE 19: 42.88636983482583
MSE 20: 44.330848212381014
MSE 21: 49.19479997258356
MSE 22: 42.07796117788765
MSE 23: 42.07174906067092
MSE 24: 40.36538402470956
MSE 25: 41.99098001251715
MSE 26: 42.32465369956007
MSE 27: 43.74441151788705
MSE 28: 39.39916560719314
MSE 29: 44.29286397545083
MSE 30: 39.46150995766847
MSE 31: 40.81387740200006
MSE 32: 31.99611291607607
MSE 33: 48.19717473891175
MSE 34: 39.203228688162895
MSE 35: 43.21730468651238
MSE 36: 39.495602950309085
MSE 37: 38.84543147287708
MSE 38: 40.916733899162814
MSE 39: 35.42555

In [75]:
comment = "Normalizing the predictors decrease standard deviation."
comment

'Normalizing the predictors decrease standard deviation.'